In [1]:
import pandas as pd
import numpy as np
import gensim
import nltk
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim import corpora, models
from datetime import datetime as dt
np.random.seed(2019)
nltk.download("wordnet")

c:\users\fazam\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fazam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
article = pd.read_json("articles.json")
article = article.sort_values(by="publication_timestamp")
article['created_at']=article['publication_timestamp'].apply(lambda x: dt.fromtimestamp(x/1000)).dt.strftime('%Y-%m')
date = article['created_at'].drop_duplicates().tolist()

In [3]:
trimestering={}
for i in date:
    if date.index(i)==28:
        trimestering[i] = int((date.index(i)-1)/4)
    else:
        trimestering[i]= int(date.index(i)/4)
article['trimester']=article['created_at'].map(trimestering)

In [4]:
trimestering

{'2017-01': 0,
 '2017-02': 0,
 '2017-03': 0,
 '2017-04': 0,
 '2017-05': 1,
 '2017-06': 1,
 '2017-07': 1,
 '2017-08': 1,
 '2017-09': 2,
 '2017-10': 2,
 '2017-11': 2,
 '2017-12': 2,
 '2018-01': 3,
 '2018-02': 3,
 '2018-03': 3,
 '2018-04': 3,
 '2018-05': 4,
 '2018-06': 4,
 '2018-07': 4,
 '2018-08': 4,
 '2018-09': 5,
 '2018-10': 5,
 '2018-11': 5,
 '2018-12': 5,
 '2019-01': 6,
 '2019-02': 6,
 '2019-03': 6,
 '2019-04': 6,
 '2019-05': 6}

In [5]:
dx = []
for i in range(0,7):
    x = article[article["trimester"]==i]
    dx.append(x)

In [6]:
dx[0].head()

,article_id,authors,contents,excerpt,fb_data,has_video,headline,image_link,keywords,li_data,...,max_velocity,pi_data,publication_timestamp,sentiment,source,topics,tw_data,velocity,created_at,trimester
3,76509020-d093-11e6-8d6f-9f4c7b0f94de,[Corazon Miller],United Airlines New Year's Day flight bound f...,A man was arrested after it was claimed he had...,"{'total_engagement_count': 1737, 'likes': 0, '...",False,Passenger arrested after United Airlines fligh...,http://media.nzherald.co.nz/webcontent/image/j...,"Business, Aviation, Airlines, Australia, Sydne...",{'li_count': 6},...,97.729933,{'pi_count': 0},1483324244514,0,"{'publisher': 'nzherald.co.nz', 'link': 'http:...","[{'id': 2, 'name': 'News'}, {'id': 28, 'name':...",{'tw_count': 100},0.115518,2017-01,0
142,b5c0ce50-d0a2-11e6-ab89-c11942898bd6,[Newstalk ZB Staff],United Airlines New Year's Day flight bound f...,A United Airlines New Year's Day flight bound ...,"{'total_engagement_count': 0, 'likes': 0, 'com...",False,United Airlines flight diverted to Auckland af...,http://www.newstalkzb.co.nz/media/16873988/uni...,,{'li_count': 0},...,0.000000,{'pi_count': 0},1483330793397,0,"{'publisher': 'newstalkzb.co.nz', 'link': 'htt...","[{'id': 2, 'name': 'News'}]",{'tw_count': 0},0.000000,2017-01,0
25,502a51e0-d0a9-11e6-ab89-c11942898bd6,[LEXI NELSON],United Airlines New Year's Day flight set to t...,"Unruly passenger goes on abusive, racist rant,...","{'total_engagement_count': 38, 'likes': 0, 'co...",False,United Airlines New Year's Day flight diverts ...,https://resources.stuff.co.nz/content/dam/imag...,"Sport, Entertainment, Business, Motoring, Worl...",{'li_count': 0},...,108.874289,{'pi_count': 0},1483333629438,0,"{'publisher': 'stuff.co.nz', 'link': 'http://s...","[{'id': 2, 'name': 'News'}, {'id': 443, 'name'...",{'tw_count': 40},-4.779850,2017-01,0
141,61489c80-d0b7-11e6-919c-7789120ed8c4,None,"You are not permitted to download, save or ema...","A man was arrested after an alleged ""racist ra...","{'total_engagement_count': 0, 'likes': 0, 'com...",False,Unruly passenger arrested in Auckland,https://www.odt.co.nz/sites/default/files/stor...,,{'li_count': 0},...,0.000000,{'pi_count': 0},1483339336000,0,"{'publisher': 'odt.co.nz', 'link': 'https://od...","[{'id': 2, 'name': 'News'}]",{'tw_count': 0},0.000000,2017-01,0
64,2ad85e20-d0c0-11e6-ab89-c11942898bd6,None,olice have arrested a badly behaved passenger ...,Police have arrested a badly behaved passenger...,"{'total_engagement_count': 3, 'likes': 0, 'com...",False,Unruly passenger forces flight to divert,http://www.radionz.co.nz/assets/news_crops/180...,"Radio New Zealand, Public Radio, News, Current...",{'li_count': 0},...,0.649721,{'pi_count': 0},1483343445250,-1,"{'publisher': 'radionz.co.nz', 'link': 'http:/...","[{'id': 2, 'name': 'News'}, {'id': 668, 'name'...",{'tw_count': 0},0.492976,2017-01,0


In [7]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [8]:
ex=[]
ex_pp=[]
for i in range(0,7):
    ex.append(pd.Series(dx[i]["excerpt"]))
    ex_pp.append(ex[i].map(preprocess))

In [9]:
co=[]
co_pp=[]
for i in range(0,7):
    co.append(pd.Series(dx[i]["contents"]))
    co_pp.append(co[i].map(preprocess))

In [10]:
dictionarye = gensim.corpora.Dictionary(ex_pp[0])

dictionarye.filter_extremes(no_below=15, no_above=0.5, keep_n=100)

bow_corpuse = [dictionarye.doc2bow(doc) for doc in ex_pp[0]]

tfidfe = models.TfidfModel(bow_corpuse)
corpus_tfidfe = tfidfe[bow_corpuse]

In [11]:
from gensim.models import CoherenceModel
for i in range(1,15):
    lda_model_tfidfe = gensim.models.LdaMulticore(corpus_tfidfe, num_topics=i, id2word=dictionarye, passes=2, workers=4)
    coherence_model_lda = CoherenceModel(model=lda_model_tfidfe, texts=ex_pp[0], dictionary=dictionarye, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.28792527305769644

Coherence Score:  0.3587360515035209

Coherence Score:  0.3268967724726705

Coherence Score:  0.36008703089039595

Coherence Score:  0.3416327324535744

Coherence Score:  0.3688391050139477

Coherence Score:  0.3878315523063306

Coherence Score:  0.3952395943361595

Coherence Score:  0.3802005820154778

Coherence Score:  0.38312025460170185

Coherence Score:  0.3655086117518152

Coherence Score:  0.37351972273588513

Coherence Score:  0.41200947607171245

Coherence Score:  0.38948746561703856


In [13]:
dictionary= []
bowcorpus= []
tfidf=[]
corpus_tfidf= []
lda_model= []
for i in range(0,7):   
    dictionaryc = gensim.corpora.Dictionary(co_pp[i])

    dictionaryc.filter_extremes(no_below=15, no_above=0.5, keep_n=100)

    bow_corpusc = [dictionaryc.doc2bow(doc) for doc in co_pp[i]]

    tfidfc = models.TfidfModel(bow_corpusc)
    corpus_tfidfc = tfidfc[bow_corpusc]
    lda_model_tfidfc= gensim.models.LdaMulticore(corpus_tfidfc, num_topics=6, id2word=dictionaryc, passes=2, workers=4)
    
    dictionary.append(dictionaryc)
    bowcorpus.append(bow_corpusc)
    tfidf.append(tfidfc)
    corpus_tfidf.append(corpus_tfidfc)
    lda_model.append(lda_model_tfidfc)

In [14]:
import pickle
import json
for i in range(0,7):
    pickle.dump(lda_model[i], open("lda_model/lda_model_{}.sav".format(i), 'wb'))
    pickle.dump(co_pp[i], open('content/content_preprocessed-{}.sav'.format(i), 'wb'))
    pickle.dump(dictionary[i], open('dictionary/dictionary-{}.sav'.format(i), 'wb'))
    pickle.dump(bowcorpus[i], open('bowcorpus/bowcorpus-{}.sav'.format(i), 'wb'))
    pickle.dump(tfidf[i], open('tfidf/tfidf-{}.sav'.format(i), 'wb'))
    pickle.dump(corpus_tfidf[i], open('corpus/corpustfidf-{}.sav'.format(i), 'wb'))

FileNotFoundError: [Errno 2] No such file or directory: 'dictionary/dictionary-0.sav'